In [22]:
# Import necessary libraries
import os
import sys
import torch
import random
import threading
import numpy as np
import pandas as pd
from pathlib import Path

# Set environment variables for GPU and device configuration
# os.environ['CUDA_VISIBLE_DEVICES'] = '5'  # Uncomment to set a different GPU
%env CUDA_VISIBLE_DEVICES=4
device = torch.device('cuda:4' if torch.cuda.is_available() else "cpu")

# Add custom paths to the Python environment
sys.path.append('./fastai1/')

# Disable RDKit warning messages
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# Import FastAI modules
from fastai import *
from fastai.text import *
from fastai.vision import *
from fastai.imports import *

# Import PyTorch and related libraries
import torch.nn.functional as F
import torchvision

# Import Scikit-learn utilities
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Display the current working directory
current_path = os.getcwd()
print(f"Current working directory: {current_path}")

# Enable interactive features for Jupyter Notebook
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [23]:
# Don't include the defalut specific token of fastai, only keep the padding token
BOS,EOS,FLD,UNK,PAD = 'xxbos','xxeos','xxfld','xxunk','xxpad'
TK_MAJ,TK_UP,TK_REP,TK_WREP = 'xxmaj','xxup','xxrep','xxwrep'
defaults.text_spec_tok = [PAD]

special_tokens = ['[BOS]', '[C@H]', '[C@@H]','[C@]', '[C@@]','[C-]','[C+]', '[c-]', '[c+]','[cH-]',
                   '[nH]', '[N+]', '[N-]', '[n+]', '[n-]' '[NH+]', '[NH2+]',
                   '[O-]', '[S+]', '[s+]', '[S-]', '[O+]', '[SH]', '[B-]','[BH2-]', '[BH3-]','[b-]',
                   '[PH]','[P+]', '[I+]',
                  '[Si]','[SiH2]', '[Se]','[SeH]', '[se]', '[Se+]', '[se+]','[te]','[te+]', '[Te]',
                  '[Pd+2]', '[Cs+]','[N@@+]','[Na+]','[OH-]','[N@]','[K+]','[F-]','[Rh]','[Ag+]','[Si]', 
                  '[Pd]', '[Cs2+]', '[Cu]', '[Cu+2]', '[Ge]', '[Sb-]', '[Cl+]', '[Cl-]', '[Br-]', '[NH4+]', 
                  '[P-]',
                  ]

class MolTokenizer(BaseTokenizer):
    def __init__(self, lang = 'en', special_tokens = special_tokens):
        self.lang = lang
        self.special_tokens = special_tokens

    def tokenizer(self, smiles):
        # add specific token '[BOS]' to represetences the start of SMILES
        smiles = '[BOS]' + smiles
        regex = '(\[[^\[\]]{1,10}\])'
        char_list = re.split(regex, smiles)
        tokens = []

        if self.special_tokens:
            for char in char_list:
                if char.startswith('['):
                    if char in special_tokens:
                        tokens.append(str(char))
                    else:
                        tokens.append('[UNK]')
                else:
                    chars = [unit for unit in char]
                    [tokens.append(i) for i in chars]

        if not self.special_tokens:
            for char in char_list:
                if char.startswith('['):
                    tokens.append(str(char))
                else:
                    chars = [unit for unit in char]
                    [tokens.append(i) for i in chars]

        #fix the 'Br' be splited into 'B' and 'r'
        if 'B' in tokens:
            for index, tok in enumerate(tokens):
                if tok == 'B':
                    if index < len(tokens)-1: # make sure 'B' is not the last character
                        if tokens[index+1] == 'r':
                            tokens[index: index+2] = [reduce(lambda i, j: i + j, tokens[index : index+2])]

        #fix the 'Cl' be splited into 'C' and 'l'
        if 'l' in tokens:
            for index, tok in enumerate(tokens):
                if tok == 'l':
                    if tokens[index-1] == 'C':
                            tokens[index-1: index+1] = [reduce(lambda i, j: i + j, tokens[index-1 : index+1])]
        return tokens

    def add_special_cases(self, toks):
        pass
    


In [24]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

In [37]:
def randomize_smiles(smiles):
    m = Chem.MolFromSmiles(smiles)
    ans = list(range(m.GetNumAtoms()))
    np.random.shuffle(ans)
    nm = Chem.RenumberAtoms(m,ans)
    return Chem.MolToSmiles(nm, canonical=False, isomericSmiles=True, kekuleSmiles=False)
from rdkit import Chem


In [38]:
def ee_smiles_augmentation(df, N_rounds, noise):
    '''
    noise: add gaussion noise to the label
    '''
    dist_aug = {col_name: [] for col_name in df}

    for i in range(df.shape[0]):
        for j in range(N_rounds):
            dist_aug['smiles'].append(randomize_smiles(df.iloc[i].smiles))
            dist_aug['Yield'].append(df.iloc[i]['Yield'] + np.random.normal(0,noise))
    print(len(dist_aug['smiles']))
    print(len(dist_aug['Yield']))
    #print(len(smiles))
    df_aug = pd.DataFrame.from_dict(dist_aug)
    df_aug = df_aug.append(df, ignore_index=True)
    return df_aug.drop_duplicates('smiles')

In [39]:
def test_smiles_augmentation(df, N_rounds):
    dist_aug = {col_name: [] for col_name in df}

    for i in range(df.shape[0]):
        for j in range(N_rounds):
            dist_aug['smiles'].append(randomize_smiles(df.iloc[i].smiles))
            dist_aug['Yield'].append(df.iloc[i]['Yield'])
    df_aug = pd.DataFrame.from_dict(dist_aug)

    return pd.DataFrame.from_dict(dist_aug)

In [40]:
random_seed(1234, True)

# Create a path to save the results
data_path = Path('/raid/aiccg/rbsunoj/supratim/ulmfit/results')
name = 'regressor1'
path = data_path / name
path.mkdir(exist_ok=True, parents=True)

In [42]:
# Define a function to process a single sheet
def process_sheet(sheet_name):
    df = pd.read_excel('./Data/Fine-tuning/866-reaction-yield-CV_1-20.xlsx', sheet_name=sheet_name)
    df = df[['smiles', 'Yield']]
    print(df)
    df['Yield'] = [int(i) for i in df['Yield']]
    
    train = df.iloc[:605, :]
    valid = df.iloc[605:692, :]
    test = df.iloc[692:, :]
    
    random_seed(1234, True)

    train_aug = ee_smiles_augmentation(train, 50, noise=0.1)
    print("Train_aug: ", train_aug.shape)
    

        
    bs = 128
    tok = Tokenizer(partial(MolTokenizer, special_tokens = special_tokens), n_cpus=6, pre_rules=[], post_rules=[])
        
    np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
    random_seed(1234, True)
        
    lm_vocab = TextLMDataBunch.from_df(path, train_aug, valid, bs=bs, tokenizer=tok,
                              chunksize=50000, text_cols=0,label_cols=1, max_vocab=60000, include_bos=False, min_freq=1, num_workers=0)
    print(f'Vocab Size: {len(lm_vocab.vocab.itos)}')
    
    pretrained_model_path = Path("./Pretraining_weights_bias/SSP1-C/models/")
    pretrained_fnames = ['C-H-activation_100_wt', 'C-H-activation_100_vocab']
    fnames = [pretrained_model_path/f'{fn}.{ext}' for fn,ext in zip(pretrained_fnames, ['pth', 'pkl'])]
        
    random_seed(1234, True)

    data_clas = TextClasDataBunch.from_df(path, train_aug, valid, bs=bs, tokenizer=tok,
                                                  chunksize=50000, text_cols='smiles',label_cols='Yield',
                                                  vocab=lm_vocab.vocab, max_vocab=60000, include_bos=False, min_freq=1, num_workers=0)
        
    print(f'Vocab Size: {len(data_clas.vocab.itos)}')
        
    random_seed(1234, True)

    lm_learner = language_model_learner(lm_vocab, AWD_LSTM, drop_mult=0.5, wd=0.00, pretrained=False)
    lm_learner = lm_learner.load_pretrained(*fnames)
    lm_learner.freeze()
    lm_learner.save_encoder(f'lm_encoder') 
        
    random_seed(1234, True)

    reg_learner = text_classifier_learner(data_clas, AWD_LSTM, pretrained=False, drop_mult=0.5, wd=0.00, metrics = [r2_score, rmse])
    reg_learner.load_encoder(f'lm_encoder')
    reg_learner.freeze()  
        
    reg_learner.model
        
    random_seed(1234, True)

    reg_learner.unfreeze()
    reg_learner.fit_one_cycle(20, 0.001, moms=(0.8,0.7))
        
    split_id = 1
    reg_learner.save(f'{split_id}_reg-866-random')
        
    preds = []

    # Randomized SMILES Predictions
    for i in range(4):
        np.random.seed(i)
        test_aug = test_smiles_augmentation(test,1)
        
        #model
        test_db = TextClasDataBunch.from_df(path, train_aug, test_aug, tokenizer=tok, vocab=lm_vocab.vocab,
                                                    text_cols='smiles', label_cols='Yield', bs=bs, include_bos=False)
        
        learner = text_classifier_learner(test_db, AWD_LSTM, pretrained=False, drop_mult=0.5, wd=0.00, metrics = [r2_score, root_mean_squared_error])
        #print(test_db)
        learner.load(f'{split_id}_reg-866-random');
        
        #get predictions
        pred,lbl = learner.get_preds(ds_type=DatasetType.Valid)
        
        preds.append(pred)
        
    # Canonical SMILES Predictions
    test_db = TextClasDataBunch.from_df(path, train_aug, test, bs=bs, tokenizer=tok,
                                      chunksize=50000, text_cols='smiles',label_cols='Yield', vocab=lm_vocab.vocab, max_vocab=60000,
                                                      include_bos=False)
        
    learner = text_classifier_learner(test_db, AWD_LSTM, pretrained=False, drop_mult=0.5, wd=0.00, metrics = [r2_score, root_mean_squared_error])
        
    learner.load(f'{split_id}_reg-866-random');
        
        
    #get predictions
    pred_canonical,lbl = learner.get_preds(ds_type=DatasetType.Valid)
        
    preds.append(pred_canonical)
    
    print('Test Set (Canonical)')
    print('RMSE:', root_mean_squared_error(pred_canonical,lbl))
    print('MAE:', mean_absolute_error(pred_canonical,lbl))
    print('R2:', r2_score(pred_canonical,lbl))
    avg_preds = sum(preds)/len(preds)
    #print('\n')
    print('Test Set (Average)')
    print('RMSE:', root_mean_squared_error(avg_preds,lbl))
    print('R2:', r2_score(avg_preds,lbl))
    print('MAE:', mean_absolute_error(avg_preds,lbl))
    
    pred1=pd.DataFrame(pred)
    def remove_tensor(x):
        return float(str(x).replace('tensor(', '').replace(')', ''))
    pred1 = pred1.applymap(remove_tensor)
    print(pred1)
        
    # Concatenate the two arrays column-wise
    concatenated_arr = np.column_stack((test, pred1))
        
    # Print the result
    print(concatenated_arr)
        
    a1=pd.DataFrame(concatenated_arr)
    #a1.to_csv('/raid/aiccg/rbsunoj/supratim/ulmfit/outputs/C-H-activation/866-result/860-test-pred-p0_' + sheet_name +'.csv')
        
# Loop through all sheet names and process each sheet
for sheet_num in range(1, 2):  # Assuming sheet names are FullCV_1, FullCV_2, ..., FullCV_20
    sheet_name = f'866-reaction-yield-CV_{sheet_num}'
    print(f"Processing sheet: {sheet_name}")
    process_sheet(sheet_name)
        
        

Processing sheet: 866-reaction-yield-CV_1
                                                smiles  Yield
0    O=S(CC1=CC(Cl)=CC=C1)(OC2=C(C#N)C=CC(OC)=C2)=O...     57
1    O=S(C1=CC(C(CC(C)C)(C#N)CC(C)C)=C(OC)C=C1)(N2[...     74
2    O=S(CC1=CC=CC=C1)(OC2=C(C#N)C=CC=C2)=O.O=C(/C=...     63
3    O=C(N(C1=CC=CC=C1F)C2=C(C#N)C=CC=C2)CCC3=CC(OC...     50
4    O=S(CC1=CC=CC=C1)(OC2=C(C#N)C=CC=C2)=O.C=CC(OC...     56
..                                                 ...    ...
861  O=S(CC1=CC(/C=C/C(OCC2=CC=C(C(C)C)C=C2)=O)=CC=...     61
862  O=C(OC1=C(C#N)C=CC(OC)=C1)CCC2=CC=CC=C2.BrC#CC...     70
863  O=C(OC1=C(C#N)C=CC=C1)CC2=CC=C(F)C=C2.C=CC(OCC...     69
864  O=S(CCC1=CC=CC=C1)(OC2=C(C#N)C=C(/C=C/C(OCC)=O...     50
865  O=C(N(C1=C(C#N)C=CC=C1)C2=CC=CC=C2C#N)CC3=C(OC...     80

[866 rows x 2 columns]
30250
30250
Train_aug:  (30855, 2)


Vocab Size: 48


Vocab Size: 48


epoch,train_loss,valid_loss,r2_score,root_mean_squared_error,time
0,4520.666016,4262.940918,-22.857170,65.291199,00:21
1,4540.906250,4095.987793,-21.922831,63.999905,00:21
2,4262.500000,4011.555420,-21.450312,63.336842,00:21
3,3658.014160,3465.394287,-18.393770,58.867596,00:20
4,2506.277344,2159.569580,-11.085839,46.471169,00:22
5,1140.107300,922.222717,-4.161137,30.368118,00:20
6,282.358124,186.087051,-0.041420,13.641373,00:21
7,81.545685,136.184265,0.237857,11.669802,00:21
8,71.569542,204.901901,-0.146715,14.314396,00:21
9,65.089149,147.578064,0.174093,12.148170,00:21


Test Set (Canonical)
RMSE: tensor(9.8875)
MAE: tensor(7.5949)
R2: tensor(0.3983)
Test Set (Average)
RMSE: tensor(9.3577)
R2: tensor(0.4610)
MAE: tensor(7.2274)
           0
0    67.3916
1    70.4851
2    49.4354
3    60.1192
4    59.3138
..       ...
169  64.1758
170  55.9711
171  73.9662
172  50.8501
173  90.1305

[174 rows x 1 columns]
[['O=C(C1=C(OC2=C(C#N)C=CC=C2)C=CC=C1)C3=CC=CC=C3.C=CC(N(C)C)=O.CC(=O)[O-].CC(=O)[O-].[Pd+2].O=C(O)[C@H](C(C)C)NC(C)=O.C(=O)([O-])[O-].[Ag+].[Ag+].OC(C(F)(F)F)C(F)(F)F.ClCCCl'
  83 67.3916]
 ['O=S(CC1=CC=CC=C1C)(OC2=C(C#N)C=CC=C2)=O.CC(OI(OC(C)=O)C1=CC=CC=C1)=O.CC(=O)[O-].CC(=O)[O-].[Pd+2].C[C@H](NC(=O)OC(C)(C)C)C(O)=O.OC(C(F)(F)F)C(F)(F)F'
  75 70.4851]
 ['O=C(OC1=C(C#N)C=C(OC)C=C1)CC2=CC=C(OC)C=C2.C=CC(OCC)=O.CC(=O)[O-].CC(=O)[O-].[Pd+2].CC(=O)NCC(=O)O.C(=O)([O-])[O-].[Ag+].[Ag+].OC(C(F)(F)F)C(F)(F)F'
  72 49.4354]
 ['O=C(C1=C(C#N)C=CC=C1)N(C)CC(CC(OC)=O)C2=CC=C(Cl)C=C2.C=CC(OCC)=O.CC(O[Pd]OC(C)=O)=O.CC(=O)NCC(=O)O.CC(=O)[O-].[Ag+].OC(C(F)(F)F)C(F)(F

In [43]:
# Define a function to process a single sheet
def process_sheet(sheet_name):
    df = pd.read_excel('/raid/aiccg/rbsunoj/supratim/ulmfit/Data/C-H-activation/866-reaction-yield-CV_1-20.xlsx', sheet_name=sheet_name)
    df = df[['smiles', 'Yield']]
    df['Yield'] = [int(i) for i in df['Yield']]
    
    train = df.iloc[:605, :]
    valid = df.iloc[605:692, :]
    test = df.iloc[692:, :]
    
    random_seed(1234, True)

    train_aug = ee_smiles_augmentation(train, 50, noise=0.6)
    print("Train_aug: ", train_aug.shape)
    

        
    bs = 128
    tok = Tokenizer(partial(MolTokenizer, special_tokens = special_tokens), n_cpus=6, pre_rules=[], post_rules=[])
        
    np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
    random_seed(1234, True)
        
    lm_vocab = TextLMDataBunch.from_df(path, train_aug, valid, bs=bs, tokenizer=tok,
                              chunksize=50000, text_cols=0,label_cols=1, max_vocab=60000, include_bos=False, min_freq=1, num_workers=0)
    print(f'Vocab Size: {len(lm_vocab.vocab.itos)}')
    
    pretrained_model_path = Path('/raid/aiccg/rbsunoj/supratim/chemdraw/pretrain3-aug-new/without_heavymetal/C-H-activation/models/')
    pretrained_fnames = ['C-H-activation_100_wt', 'C-H-activation_100_vocab']
    fnames = [pretrained_model_path/f'{fn}.{ext}' for fn,ext in zip(pretrained_fnames, ['pth', 'pkl'])]
        
    random_seed(1234, True)

    data_clas = TextClasDataBunch.from_df(path, train_aug, valid, bs=bs, tokenizer=tok,
                                                  chunksize=50000, text_cols='smiles',label_cols='Yield',
                                                  vocab=lm_vocab.vocab, max_vocab=60000, include_bos=False, min_freq=1, num_workers=0)
        
    print(f'Vocab Size: {len(data_clas.vocab.itos)}')
        
    random_seed(1234, True)

    lm_learner = language_model_learner(lm_vocab, AWD_LSTM, drop_mult=0.5, wd=0.00, pretrained=False)
    lm_learner = lm_learner.load_pretrained(*fnames)
    lm_learner.freeze()
    lm_learner.save_encoder(f'lm_encoder') 
        
    random_seed(1234, True)

    reg_learner = text_classifier_learner(data_clas, AWD_LSTM, pretrained=False, drop_mult=0.5, wd=0.00, metrics = [r2_score, rmse])
    reg_learner.load_encoder(f'lm_encoder')
    reg_learner.freeze()  
        
    reg_learner.model
        
    random_seed(1234, True)

    reg_learner.unfreeze()
    reg_learner.fit_one_cycle(20, 0.001, moms=(0.8,0.7))
        
    split_id = 1
    reg_learner.save(f'{split_id}_reg-866-random')
        
    preds = []

    # Randomized SMILES Predictions
    for i in range(4):
        np.random.seed(i)
        test_aug = test_smiles_augmentation(test,1)
        
        #model
        test_db = TextClasDataBunch.from_df(path, train_aug, test_aug, tokenizer=tok, vocab=lm_vocab.vocab,
                                                    text_cols='smiles', label_cols='Yield', bs=bs, include_bos=False)
        
        learner = text_classifier_learner(test_db, AWD_LSTM, pretrained=False, drop_mult=0.5, wd=0.00, metrics = [r2_score, root_mean_squared_error])
        #print(test_db)
        learner.load(f'{split_id}_reg-866-random');
        
        #get predictions
        pred,lbl = learner.get_preds(ds_type=DatasetType.Valid)
        
        preds.append(pred)
        
    # Canonical SMILES Predictions
    test_db = TextClasDataBunch.from_df(path, train_aug, test, bs=bs, tokenizer=tok,
                                      chunksize=50000, text_cols='smiles',label_cols='Yield', vocab=lm_vocab.vocab, max_vocab=60000,
                                                      include_bos=False)
        
    learner = text_classifier_learner(test_db, AWD_LSTM, pretrained=False, drop_mult=0.5, wd=0.00, metrics = [r2_score, root_mean_squared_error])
        
    learner.load(f'{split_id}_reg-866-random');
        
        
    #get predictions
    pred_canonical,lbl = learner.get_preds(ds_type=DatasetType.Valid)
        
    preds.append(pred_canonical)
    
    print('Test Set (Canonical)')
    print('RMSE:', root_mean_squared_error(pred_canonical,lbl))
    print('MAE:', mean_absolute_error(pred_canonical,lbl))
    print('R2:', r2_score(pred_canonical,lbl))
    avg_preds = sum(preds)/len(preds)
    #print('\n')
    print('Test Set (Average)')
    print('RMSE:', root_mean_squared_error(avg_preds,lbl))
    print('R2:', r2_score(avg_preds,lbl))
    print('MAE:', mean_absolute_error(avg_preds,lbl))
    
    pred1=pd.DataFrame(pred)
    def remove_tensor(x):
        return float(str(x).replace('tensor(', '').replace(')', ''))
    pred1 = pred1.applymap(remove_tensor)
    print(pred1)
        
    # Concatenate the two arrays column-wise
    concatenated_arr = np.column_stack((test, pred1))
        
    # Print the result
    print(concatenated_arr)
        
    a1=pd.DataFrame(concatenated_arr)
    #a1.to_csv('/raid/aiccg/rbsunoj/supratim/ulmfit/outputs/C-H-activation/866-result/860-test-pred-p0_' + sheet_name +'.csv')
        
# Loop through all sheet names and process each sheet
# Loop through all sheet names and process each sheet
for sheet_num in range(1, 2):  # Assuming sheet names are FullCV_1, FullCV_2, ..., FullCV_20
    sheet_name = f'866-reaction-yield-CV_{sheet_num}'
    print(f"Processing sheet: {sheet_name}")
    process_sheet(sheet_name)
        

Processing sheet: 866-reaction-yield-CV_1
30250
30250
Train_aug:  (30855, 2)


Vocab Size: 48


Vocab Size: 48


epoch,train_loss,valid_loss,r2_score,root_mean_squared_error,time
0,4521.926270,4296.550293,-23.045261,65.548073,00:21
1,4545.556152,4110.928711,-22.006445,64.116524,00:21
2,4275.340332,4137.287109,-22.153957,64.321747,00:21
3,3676.725098,3278.531494,-17.348009,57.258465,00:21
4,2528.989990,2053.898438,-10.494459,45.319958,00:22
5,1163.950439,896.779480,-4.018746,29.946276,00:21
6,309.430267,196.110367,-0.097514,14.003942,00:21
7,113.489227,126.111160,0.294230,11.229922,00:22
8,101.285965,188.988419,-0.057657,13.747306,00:21
9,97.228401,140.403000,0.214247,11.849177,00:21


Test Set (Canonical)
RMSE: tensor(10.4147)
MAE: tensor(8.4032)
R2: tensor(0.3324)
Test Set (Average)
RMSE: tensor(9.5953)
R2: tensor(0.4333)
MAE: tensor(7.5862)
           0
0    61.7026
1    68.7779
2    52.5514
3    56.7865
4    74.0658
..       ...
169  53.5452
170  59.4516
171  72.2125
172  67.1033
173  88.3788

[174 rows x 1 columns]
[['O=C(C1=C(OC2=C(C#N)C=CC=C2)C=CC=C1)C3=CC=CC=C3.C=CC(N(C)C)=O.CC(=O)[O-].CC(=O)[O-].[Pd+2].O=C(O)[C@H](C(C)C)NC(C)=O.C(=O)([O-])[O-].[Ag+].[Ag+].OC(C(F)(F)F)C(F)(F)F.ClCCCl'
  83 61.7026]
 ['O=S(CC1=CC=CC=C1C)(OC2=C(C#N)C=CC=C2)=O.CC(OI(OC(C)=O)C1=CC=CC=C1)=O.CC(=O)[O-].CC(=O)[O-].[Pd+2].C[C@H](NC(=O)OC(C)(C)C)C(O)=O.OC(C(F)(F)F)C(F)(F)F'
  75 68.7779]
 ['O=C(OC1=C(C#N)C=C(OC)C=C1)CC2=CC=C(OC)C=C2.C=CC(OCC)=O.CC(=O)[O-].CC(=O)[O-].[Pd+2].CC(=O)NCC(=O)O.C(=O)([O-])[O-].[Ag+].[Ag+].OC(C(F)(F)F)C(F)(F)F'
  72 52.5514]
 ['O=C(C1=C(C#N)C=CC=C1)N(C)CC(CC(OC)=O)C2=CC=C(Cl)C=C2.C=CC(OCC)=O.CC(O[Pd]OC(C)=O)=O.CC(=O)NCC(=O)O.CC(=O)[O-].[Ag+].OC(C(F)(F)F)C(F)(